<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Practice SQL with Pandas pt. 2

---


We've learned about relational databases and the language most used to query them, SQL.  

In this lab we are going to gain more practice converting information to a SQL DB, querying the data and then analysing it with Python.

In [1]:
# Necessary Libraries
import pandas as pd
import sqlite3
from pandas.io import sql

### 1.  Read in the EuroMart CSV Data
- 'EuroMart-ListOfOrders.csv'
- 'EuroMart-OrderBreakdown.csv'
- 'EuroMart-SalesTargets.csv'

In [2]:
# Reading CSV to Dataframe
orders = pd.read_csv('../datasets/csv/EuroMart-ListOfOrders.csv', encoding = 'utf-8')
OBD =  pd.read_csv('../datasets/csv/EuroMart-OrderBreakdown.csv', encoding = 'utf-8')
sales_targets =  pd.read_csv('../datasets/csv/EuroMart-SalesTargets.csv', encoding = 'utf-8')

### 2. Rename columns to remove any spaces

In [3]:
# renaming columns to remove spaces.
orders.columns = ['order_id','order_date','customer_name','city','country','region',
                        'segment','ship_date','ship_mode','state']
OBD.columns = ['order_id','product_name','discount','sales','profit','quantity',
          'category','sub-category']
 
sales_targets.columns = ['month_of_order_date','category','target']

### 3. Remove dollar signs from sales and profit columns in the order breakdown dataframe.

Convert the columns to float.

In [4]:
OBD.head()

,order_id,product_name,discount,sales,profit,quantity,category,sub-category
0,BN-2011-7407039,"Enermax Note Cards, Premium",0.5,$45.00,-$26.00,3,Office Supplies,Paper
1,AZ-2011-9050313,"Dania Corner Shelving, Traditional",0.0,$854.00,$290.00,7,Furniture,Bookcases
2,AZ-2011-6674300,"Binney & Smith Sketch Pad, Easy-Erase",0.0,$140.00,$21.00,3,Office Supplies,Art
3,BN-2011-2819714,"Boston Markers, Easy-Erase",0.5,$27.00,-$22.00,2,Office Supplies,Art
4,BN-2011-2819714,"Eldon Folders, Single Width",0.5,$17.00,-$1.00,2,Office Supplies,Storage


In [5]:
OBD['sales'][OBD['sales'].str.contains(',')].head()

12     $1,384.00
117    $4,544.00
128    $1,003.00
140    $2,076.00
153    $1,314.00
Name: sales, dtype: object

In [6]:
# removing the dollar signs from sales and profit
OBD['sales'] = OBD['sales'].map(lambda x: x.strip('$'))
OBD['sales'] = OBD['sales'].map(lambda x: float(x.replace(',','')))

OBD['profit'] = OBD['profit'].map(lambda x: x.replace('$',''))
OBD['profit'] = OBD['profit'].map(lambda x: float(x.replace(',','')))

### 4. Create a SQL Database called 'EuroMart' and save the three dataframes as SQL tables. 

In [7]:
# Establishing Local DB connection
db_connection = sqlite3.connect('../datasets/EuroMart.db.sqlite')

# # Reading out DataFrames as SQL Table
orders.to_sql(name = 'orders', con = db_connection, if_exists = 'replace', index = False)
OBD.to_sql(name = 'order_breakdown', con = db_connection, if_exists = 'replace', index = False)
sales_targets.to_sql(name = 'sales_targets', con = db_connection, if_exists = 'replace', index = False)

In [8]:
# Getting the Column Labels.  
orders.head(1)

,order_id,order_date,customer_name,city,country,region,segment,ship_date,ship_mode,state
0,BN-2011-7407039,1/1/2011,Ruby Patel,Stockholm,Sweden,North,Home Office,1/5/2011,Economy Plus,Stockholm


In [9]:
OBD.head(1)

,order_id,product_name,discount,sales,profit,quantity,category,sub-category
0,BN-2011-7407039,"Enermax Note Cards, Premium",0.5,45.0,-26.0,3,Office Supplies,Paper


In [10]:
sales_targets.head(1)

,month_of_order_date,category,target
0,Jan-11,Furniture,"$10,000.00"


### 5. How many orders has each Customer placed? 

In [11]:
# Getting all customer names and setting them to a pandas object
customers = sql.read_sql('SELECT customer_name FROM orders', con = db_connection)

# counting unique values in the list
customers['customer_name'].value_counts().head()

Jose Gambino       13
Mark Washington    12
Kayla Tearle       12
Aaron Bootman      11
Lori Miller        11
Name: customer_name, dtype: int64

In [12]:
# pure SQL
sql.read_sql('SELECT customer_name, count(customer_name) as count ' 
             'FROM orders '
             'WHERE customer_name like "A%" '
             'GROUP BY customer_name '
             'HAVING count<10 '
             'ORDER BY count DESC', 
             con = db_connection).head(30)

,customer_name,count
0,Audrey Knowles,9
1,Anna Macdonald,9
2,Angus Reading,9
3,Angus De Groot,9
4,Amy Nixon,9
5,Alannah Langridge,9
6,Archer Hort,8
7,Angelina Hurst,8
8,Alexis Code,8
9,Aidan Rowe,8


In [13]:
# pure SQL
sql.read_sql('SELECT customer_name, count(customer_name) as count ' 
             'FROM orders '
             'GROUP BY customer_name '
             'ORDER BY count DESC', 
             con = db_connection, 
             index_col='customer_name').head()

,count
customer_name,
Jose Gambino,13
Mark Washington,12
Kayla Tearle,12
Terence Welch,11
Rachel Tyler,11


> *If you're doubting your output check using Pandas*

### 6. Create a Query to return a table of only geographic features from the List of Orders Table.

In [14]:
# City, Country, Region and State are Geographic, Yeah?
sql.read_sql('SELECT DISTINCT city, country, region, state '
             'FROM orders '
             'ORDER BY country', 
             con = db_connection).head(30)

,city,country,region,state
0,Vienna,Austria,Central,Vienna
1,Linz,Austria,Central,Upper Austria
2,Innsbruck,Austria,Central,Tyrol
3,Graz,Austria,Central,Styria
4,Salzburg,Austria,Central,Salzburg
5,Villach,Austria,Central,Carinthia
6,Klagenfurt,Austria,Central,Carinthia
7,Wels,Austria,Central,Upper Austria
8,Leuven,Belgium,Central,Flemish Brabant
9,Namur,Belgium,Central,Namur


### 7. Create a Query to return a table with all of the orders that had a negative profit from the Order Breakdown Table.

In [15]:
sql.read_sql('SELECT * ' 
             'FROM order_breakdown '
             'WHERE profit<0', 
             con = db_connection).head()

,order_id,product_name,discount,sales,profit,quantity,category,sub-category
0,BN-2011-7407039,"Enermax Note Cards, Premium",0.5,45.0,-26.0,3,Office Supplies,Paper
1,BN-2011-2819714,"Boston Markers, Easy-Erase",0.5,27.0,-22.0,2,Office Supplies,Art
2,BN-2011-2819714,"Eldon Folders, Single Width",0.5,17.0,-1.0,2,Office Supplies,Storage
3,BN-2011-3248724,"Ikea Classic Bookcase, Metal",0.6,987.0,-1012.0,6,Furniture,Bookcases
4,BN-2011-3248724,"Binney & Smith Sketch Pad, Blue",0.5,116.0,-56.0,5,Office Supplies,Art


In [16]:
# Alternative way:
# Pretty much identifying any cell in the profit column that has a '-' sign
sql.read_sql('SELECT * '
             'FROM order_breakdown '
             'WHERE profit LIKE "%-%"', 
             con = db_connection).head()
# Had not converted values ints prior to writing this.  
# Works with ints and objects!!!

,order_id,product_name,discount,sales,profit,quantity,category,sub-category
0,BN-2011-7407039,"Enermax Note Cards, Premium",0.5,45.0,-26.0,3,Office Supplies,Paper
1,BN-2011-2819714,"Boston Markers, Easy-Erase",0.5,27.0,-22.0,2,Office Supplies,Art
2,BN-2011-2819714,"Eldon Folders, Single Width",0.5,17.0,-1.0,2,Office Supplies,Storage
3,BN-2011-3248724,"Ikea Classic Bookcase, Metal",0.6,987.0,-1012.0,6,Furniture,Bookcases
4,BN-2011-3248724,"Binney & Smith Sketch Pad, Blue",0.5,116.0,-56.0,5,Office Supplies,Art


### 8. Construct a query to return a table with the Customer Name and Product Name.  

> **Note:** This will require a join!

In [17]:
# So we're like doing a thing
sql.read_sql('SELECT orders.order_id, orders.customer_name, order_breakdown.product_name '
             'FROM orders '
             'LEFT JOIN order_breakdown '
             'ON orders.order_id = order_breakdown.order_id ',
            con = db_connection,index_col='customer_name').head()


,order_id,product_name
customer_name,,
Ruby Patel,BN-2011-7407039,"Enermax Note Cards, Premium"
Summer Hayward,AZ-2011-9050313,"Dania Corner Shelving, Traditional"
Devin Huddleston,AZ-2011-6674300,"Binney & Smith Sketch Pad, Easy-Erase"
Mary Parker,BN-2011-2819714,"Boston Markers, Easy-Erase"
Mary Parker,BN-2011-2819714,"Eldon Folders, Single Width"


### 9.  How many orders for "Office Supplies" (Category) has Sweden made?

> **Note:** from this point on you'll probably be combining SQL and Pandas, in that you would use SQL querys to gather the relevant information and use Pandas to analyze it.

In [18]:
swedish_supplies = sql.read_sql(
            'SELECT orders.order_id, orders.country, order_breakdown.category '            
            'FROM orders '
            'LEFT JOIN order_breakdown '
            'ON orders.order_id = order_breakdown.order_id '
            'WHERE orders.country = "Sweden" and order_breakdown.category = "Office Supplies"',
            con = db_connection,index_col='country')

len(swedish_supplies)

133

In [19]:
swedish_supplies.head()

,order_id,category
country,,
Sweden,BN-2011-7407039,Office Supplies
Sweden,BN-2011-2807470,Office Supplies
Sweden,BN-2011-7087921,Office Supplies
Sweden,BN-2011-827720,Office Supplies
Sweden,BN-2011-8234232,Office Supplies


### 10. What was the total sales for products that have been discounted? 

In [20]:
discount_sales = sql.read_sql(
                'SELECT discount, sales '
                'FROM order_breakdown '
                'WHERE discount > 0',
                con = db_connection)

discount_sales['sales'].sum()

1115614.0

In [21]:
# pure sql
discount_sales_sum = sql.read_sql(
            'SELECT sum(sales) as total_sales '
            'FROM order_breakdown '
            'WHERE discount > 0',
            con = db_connection,index_col='total_sales')
discount_sales_sum

""
total_sales
1115614.0


### 11. What is the total quantity of objects sold for each country?

In [22]:
order_counts = sql.read_sql(
                'SELECT order_breakdown.quantity, orders.country '
                'FROM orders '
                'INNER JOIN order_breakdown '
                'ON orders.order_id = order_breakdown.order_id ',
                con = db_connection)

order_counts.groupby('country').sum()

,quantity
country,
Austria,973
Belgium,532
Denmark,204
Finland,201
France,7329
Germany,6179
Ireland,392
Italy,3612
Netherlands,1526


In [23]:
# pure sql
order_counts_grouped = sql.read_sql(
                'SELECT sum(order_breakdown.quantity) as total_quantity,'
                'orders.country '
                'FROM orders '
                'INNER JOIN order_breakdown '
                'ON orders.order_id = order_breakdown.order_id '
                'GROUP BY orders.country',
                con = db_connection,index_col='country')

order_counts_grouped

,total_quantity
country,
Austria,973
Belgium,532
Denmark,204
Finland,201
France,7329
Germany,6179
Ireland,392
Italy,3612
Netherlands,1526


### 12. In what Countries are profits lowest? (Report lowest 5-10)

In [24]:
# gather country and profits. 
profits = sql.read_sql(
                'SELECT order_breakdown.profit, orders.country '
                'FROM orders '
                'INNER JOIN order_breakdown '
                'ON orders.order_id = order_breakdown.order_id ',
            con = db_connection)

# Groupby country and sum with sort on profits
profits.groupby('country').sum().sort_values(by='profit').reset_index()

,country,profit
0,Netherlands,-37188.0
1,Sweden,-17524.0
2,Portugal,-8704.0
3,Ireland,-6886.0
4,Denmark,-3608.0
5,Finland,3908.0
6,Norway,5167.0
7,Switzerland,7234.0
8,Belgium,9912.0
9,Italy,15802.0


In [25]:
# pure sql
profits = sql.read_sql(
                'SELECT  orders.country, sum(order_breakdown.profit) as total_profit '
                'FROM orders '
                'INNER JOIN order_breakdown '
                'ON orders.order_id = order_breakdown.order_id '
                'GROUP BY orders.country '
                'ORDER BY total_profit',
            con = db_connection,index_col='country')
profits

,total_profit
country,
Netherlands,-37188.0
Sweden,-17524.0
Portugal,-8704.0
Ireland,-6886.0
Denmark,-3608.0
Finland,3908.0
Norway,5167.0
Switzerland,7234.0
Belgium,9912.0


### 13. What Countries have the best and worst profit to sales ratios?

Total profits divided by total sales -
this is saying for every dollar of product sold, how much is the profit.

In [26]:
#Grabbing Profits, Sales and Countries
spr = sql.read_sql('SELECT order_breakdown.profit, '
                   'order_breakdown.sales, orders.country '
                   'FROM orders '
                   'INNER JOIN order_breakdown '
                   'ON orders.order_id = order_breakdown.order_id ',
            con = db_connection)

# Summing profits and sales by Country
spr2 = spr.groupby('country').sum().sort_values(by='profit')

# creating the ratio column
spr2['ratio'] = spr2['profit']/spr2['sales']

# Sorting by ratio column
spr2.sort_values(by='ratio', ascending = False)

,profit,sales,ratio
country,,,
Switzerland,7234.0,24874.0,0.290826
Austria,21332.0,79382.0,0.268726
Norway,5167.0,20529.0,0.251693
Belgium,9912.0,42320.0,0.234216
United Kingdom,90382.0,420497.0,0.214941
Finland,3908.0,20702.0,0.188774
Spain,47067.0,249402.0,0.188719
Germany,86279.0,488681.0,0.176555
France,70067.0,609683.0,0.114924


In [27]:
# pure sql
spr = sql.read_sql('SELECT orders.country,'
                   'sum(order_breakdown.profit) as total_profit, '
                   'sum(order_breakdown.sales) as total_sales, '
                   '(sum(order_breakdown.profit) / sum(order_breakdown.sales)) as ratio '
                   'FROM orders '
                   'INNER JOIN order_breakdown '
                   'ON orders.order_id = order_breakdown.order_id '
                   'GROUP BY orders.country '
                   'ORDER BY ratio DESC',
            con = db_connection,index_col='country')
spr

,total_profit,total_sales,ratio
country,,,
Switzerland,7234.0,24874.0,0.290826
Austria,21332.0,79382.0,0.268726
Norway,5167.0,20529.0,0.251693
Belgium,9912.0,42320.0,0.234216
United Kingdom,90382.0,420497.0,0.214941
Finland,3908.0,20702.0,0.188774
Spain,47067.0,249402.0,0.188719
Germany,86279.0,488681.0,0.176555
France,70067.0,609683.0,0.114924


### 14. What Shipping method is most common for 'Bookcases' (Sub Category)

In [28]:
sql.read_sql('SELECT orders.ship_mode, order_breakdown."sub-category" '
             'FROM orders '
             'INNER JOIN order_breakdown '
             'ON orders.order_id = order_breakdown.order_id '
             'WHERE "sub-category" = "Bookcases" ' ,
            con = db_connection)['ship_mode'].value_counts()

Economy         234
Economy Plus     76
Priority         59
Immediate        22
Name: ship_mode, dtype: int64

In [29]:
# pure sql
sql.read_sql('SELECT orders.ship_mode, count(orders.ship_mode) as ship_mode_counts '
             'FROM orders '
             'INNER JOIN order_breakdown '
             'ON orders.order_id = order_breakdown.order_id '
             'WHERE "sub-category" = "Bookcases" '
             'GROUP BY orders.ship_mode '
             'ORDER BY ship_mode_counts DESC',
            con = db_connection,index_col='ship_mode')

,ship_mode_counts
ship_mode,
Economy,234
Economy Plus,76
Priority,59
Immediate,22


### 15 .What city in the Orders table generated the highest net sales?  (List all the cities and countries in descending order by net sales.)

In [30]:
sql.read_sql('SELECT orders.city, orders.country, ' 
             'order_breakdown.sales '
             'FROM orders '
             'INNER JOIN order_breakdown '
             'ON orders.order_id = order_breakdown.order_id ',
            con = db_connection).groupby(['city','country']
                ).sum().sort_values(by='sales', ascending = False).head(10)

,,sales
city,country,
London,United Kingdom,69230.0
Berlin,Germany,52555.0
Vienna,Austria,51844.0
Madrid,Spain,44981.0
Paris,France,42245.0
Rome,Italy,28330.0
Barcelona,Spain,27405.0
Hamburg,Germany,23574.0
Marseille,France,21677.0


In [31]:
# pure sql
cities = sql.read_sql(
             'SELECT orders.city, orders.country, ' 
             'sum(order_breakdown.sales) as total_sales '
             'FROM orders '
             'INNER JOIN order_breakdown '
             'ON orders.order_id = order_breakdown.order_id '
             'GROUP BY orders.city, orders.country '
             'ORDER BY total_sales DESC '
             ,
            con = db_connection, index_col='city')

cities[:20]

,country,total_sales
city,,
London,United Kingdom,69230.0
Berlin,Germany,52555.0
Vienna,Austria,51844.0
Madrid,Spain,44981.0
Paris,France,42245.0
Rome,Italy,28330.0
Barcelona,Spain,27405.0
Hamburg,Germany,23574.0
Marseille,France,21677.0
